In [150]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [151]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [152]:
spaces=[]
spaces = snapshot.Query.spaces(
  orderBy='id',
  orderDirection='desc',
  first=1000,
)
spaces_list = sg.query_df([
    spaces.id
    ]
  )

In [153]:
spaces2 = []
n=0
t=2
while n<1:
  spaces2 = snapshot.Query.spaces(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t
  )
  spaces_list2 = sg.query_df([
    spaces2.id
    ]
  )
  spaces_list = pd.concat([spaces_list,spaces_list2])
  t=t+1
  l=len(spaces_list2)
  print(t,l)
  if len(spaces_list2)<1000: n=1

olympus_coord=spaces_list.index[spaces_list['spaces_id'] == 'olympusdao.eth'].tolist()


3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 963


In [154]:
##spaces_list

In [166]:
numspaces = len(spaces_list)
numspaces

7963

In [167]:
##spaces_list = pd.DataFrame(spaces_list)
##
##spaces_list.iat[coordinate,0]

In [177]:
space_name = spaces_list.iat[1,0]
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

proposals_snapshots.insert(0,'dao',space_name)

proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,10.096352,10.096352
1,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,0.000000,10.096352
2,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.498913,0.498913
3,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.000000,0.498913
4,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,2.711773,2.711773
5,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,0.000000,2.711773
6,🍯dao.eth,Test,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa,Test,2.275148,5.901114
7,🍯dao.eth,Test,QmZQkuw2DFuut5FdYrDji2fKoHW3rWbQd9TbcWgDSufoHa,Test,3.625966,5.901114
8,🍯dao.eth,Donate 1 BAL on Gitcoin,QmV7WfE1vxPdwNYoom57zTRBVB38hWRPCZJgXEgD2zxK65,Test a donation of 1Bal to the reality.eth Git...,2.476520,2.476520
9,🍯dao.eth,Donate 1 BAL on Gitcoin,QmV7WfE1vxPdwNYoom57zTRBVB38hWRPCZJgXEgD2zxK65,Test a donation of 1Bal to the reality.eth Git...,0.000000,2.476520


In [187]:
space_name = spaces_list.iat[2,0]

proposals2 = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots2 = sg.query_df([
    proposals2.title,
    proposals2.id,
    proposals2.body,
    proposals2.scores,
    proposals2.scores_total
])

proposals_snapshots2.insert(0,'dao',space_name)

proposals_snapshots2

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.00000,0.00000
1,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.00000,0.00000
2,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.00000,0.00000
3,안철수.eth,안철수 국민의당 대선 후보 공약,0x53d7118740830b2186d271177661b5d4e98c6b9dcef2...,\n[G5 경제강국 대한민국 5.5.5 신성장 전략]\n\n1. 정부조직 개편\n ...,0.00000,1.00923
4,안철수.eth,안철수 국민의당 대선 후보 공약,0x53d7118740830b2186d271177661b5d4e98c6b9dcef2...,\n[G5 경제강국 대한민국 5.5.5 신성장 전략]\n\n1. 정부조직 개편\n ...,1.00923,1.00923


In [196]:
proposals3 = []
t=3
while t<=numspaces :
  space_name = spaces_list.iat[t,0]
  proposals3 = snapshot.Query.proposals(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t,
    where=[
      snapshot.Proposal.space == space_name,
      snapshot.Proposal.state == 'closed'
      ]
  )
  proposals_snapshots3 = sg.query_df([
    proposals3.title,
    proposals3.id,
    proposals3.body,
    proposals3.scores,
    proposals3.scores_total
])
  proposals_snapshots3.insert(0,'dao',space_name)
  proposals_snapshots = pd.concat([proposals_snapshots,proposals_snapshots2,proposals_snapshots3])
  t=t+1
  l=len(proposals_snapshots3)
  if t%10 == 0: print (t,space_name,l)
  if len(proposals_snapshots3)>0:
    print(t,space_name,l)

proposals_snapshots

10 狂奔的蜗牛5.eth 0
20 深圳刘先生.eth 0
30 掌上中药网平台.eth 0
40 商丘工学院.eth 0
50 zzrui.eth 0
60 zys888.eth 0
70 zsmjcaib.eth 0
80 zoogangdao.eth 0
90 zombietoadzdao.eth 0
100 zloco.eth 0
110 zipdao.eth 0
120 zhupp.eth 0
130 zhongzy.eth 0
140 zhejiangshuilishuidianxueyuan.eth 0
150 zhangwill.eth 0
160 zeusdao.eth 0
170 zerion1.eth 0
180 zeldala.eth 0
190 zdao-wilderwheels.eth 0
200 zaoniao.eth 0
210 yyygames.eth 0
220 yuricat.eth 0
230 yuihatano.eth 0
240 youzufei.eth 0
250 yotta21.eth 0
260 yokaihouse.eth 0
270 yjsoly.eth 0
280 yinfinance.eth 0
290 yieldfarming-insure 0
300 yffs 0
310 yeld 0
320 ycggo.eth 0
330 yaphets.eth 0
340 yangzn.eth 0
350 yanabu.eth 0
360 yachovcohen.eth 0
370 xwg.eth 0
380 xuemingxin.eth 0
390 xrpchain.eth 0
400 xmtplabs.eth 0
410 xinyang.eth 0
420 xillion.eth 0
430 xiaowukong.eth 0
440 xiaoj.eth 0
450 xiami.eth 0
460 xf0707.eth 0
470 xddcyy.eth 0
480 xbing.eth 0
490 wyzwang168.eth 0
500 wujuanliwu.eth 0
510 wscrj.eth 0
520 woxapp.eth 0
530 worldofme.eth 0
540 woodies.eth 0
5

C:\Users\ianfe\PycharmProjects\pythonProject\venv\lib\site-packages\subgrounds\subgraph\fieldpath.py:485: ResourceWarning: unclosed <socket.socket fd=3600, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.56', 64012), raddr=('104.26.13.125', 443)>
  match super().__getattribute__(__name):


1610 drawdao.eth 0
1620 axiba.eth 0
1630 timepieceapes.eth 0
1640 megavega.eth 0
1650 matrixdaoresearch.eth 0
1660 football3.eth 0
1670 rizzledao.eth 0
1680 adventurepizza.eth 0
1690 medievaldao.eth 0
1700 pltlotto.eth 0
1710 skeletonzgang.eth 0
1720 niumcrypto.eth 0
1730 nandemotoken.eth 0
1740 nagadao.eth 0
1750 sentineldao.eth 0
1760 wrise.eth 0
1770 shanhe.eth 0
1780 notmadi.eth 0
1790 678dao.eth 0
1800 dixelclub.eth 0
1810 nostalgiadao.eth 0
1820 kitkwan.eth 0
1830 fatexdao.eth 0
1840 xgzszdfs.eth 0
1850 accelone.eth 0
1860 secondskin.eth 0
1870 binamongov.eth 0
1880 web3venture.eth 0
1890 nunababy.eth 0
1900 grugslair.eth 0
1910 xytothemoon.eth 0
1920 bentokenfinance.eth 0
1930 gonewind.eth 0
1940 aqualisdao.eth 0
1950 civicsdaotest.eth 0
1960 55556.eth 0
1970 bemular.eth 0
1980 mmfinance.eth 0
1990 apeznthehood.eth 0
2000 hongtianyou.eth 0
2010 shipchain.eth 0
2020 desiege.eth 0
2030 createit.eth 0
2040 alohafrens.eth 0
2050 cryptomakersfoundation.eth 0
2060 thekrok.eth 0
2070 s

KeyError: 'errors'

In [197]:
proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,10.096352,10.096352
1,🍯dao.eth,Should I get 2 times one ETH,QmQ1s8FHhX2FtFGRcfbbdJmT1sU8EwPymDytsHyzFp5YoL,Should I get 2 times one ETH?,0.000000,10.096352
2,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.498913,0.498913
3,🍯dao.eth,What is the next achievement we should fund be...,QmZwnht7pRE2xGnUoQMU37eoNV8eEnobrejPZwDzdpBjaY,,0.000000,0.498913
4,🍯dao.eth,Should the 🍯️DAO set its Gnosis Safe as the co...,QmWwo3TkBtUybd7wcwix6yuGtApR427FBCxiSGJNwowDoN,This is a proposal to set the controller of th...,2.711773,2.711773
...,...,...,...,...,...,...
0,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.000000,0.000000
1,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.000000,0.000000
2,안철수.eth,안철수 후보의 가상 자산 관련 공약,0x4b51e3f70647398ba7a7799fcbc10cccf34e7a3fc165...,\n안철수 후보가 20대 대통령이 된다면\n가상자산 업계는 어떻게 될 것이라고 예상...,0.000000,0.000000
3,안철수.eth,안철수 국민의당 대선 후보 공약,0x53d7118740830b2186d271177661b5d4e98c6b9dcef2...,\n[G5 경제강국 대한민국 5.5.5 신성장 전략]\n\n1. 정부조직 개편\n ...,0.000000,1.009230


In [198]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
  )


In [199]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [ ]:
##placing 25 options just in case. Could be more, but I hope not
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22','option_24','option_24','option_25])

In [ ]:
dao_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [ ]:
dao_governance_view